In [2]:
import pandas as pd
from modeling import LightningBinConv
from data import TSDataset, TSDatasetTest
from torch.utils.data import DataLoader
from preprocessing import get_preprocessing_pipeline
import lightning.pytorch as pl
import matplotlib.pyplot as plt

In [3]:
# download toy dataset to validate the methods
df = pd.read_csv(
    "https://raw.githubusercontent.com/AileenNielsen/"
    "TimeSeriesAnalysisWithPython/master/data/AirPassengers.csv",
    index_col=0,
    parse_dates=True,
)
df.head()

,#Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


In [177]:
#hyperparameters
prediction_length = 12
context_length = 12
num_bins = 5000
min_value = -3
max_value = 3
offset = -36 # for train/test split

In [178]:
train_data = df.values[:offset]
test_data = df.values[offset-context_length:]

In [179]:
pipeline = get_preprocessing_pipeline()

In [180]:
net = LightningBinConv(
    context_length=context_length,
    num_bins=num_bins,
    kernel_size_across_bins_2d=101,
    kernel_size_across_bins_1d=101,
    num_filters_2d= 8,
    num_filters_1d= 32,
    is_cum_sum = True
)

In [181]:
dataset = TSDataset(data=train_data, context_length=context_length, scaler=pipeline)
train_loader = DataLoader(dataset, batch_size=int(1e6), shuffle=True)

In [182]:
net

LightningBinConv(
  (conv): Conv2d(1, 8, kernel_size=(12, 101), stride=(1, 1))
  (conv1d_1): Conv1d(8, 32, kernel_size=(101,), stride=(1,))
  (conv1d_2): Conv1d(32, 1000, kernel_size=(101,), stride=(1,))
)

In [ ]:
trainer = pl.Trainer(max_epochs=300)
trainer.fit(net, train_loader)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name     | Type   | Params | Mode 
--------------------------------------------
0 | conv     | Conv2d | 9.7 K  | train
1 | conv1d_1 | Conv1d | 25.9 K | train
2 | conv1d_2 | Conv1d | 3.2 M  | train
--------------------------------------------
3.3 M     Trainable params
0         Non-trainable params
3.3 M     Total params
13.074    Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode
/Users/andreichernov/miniforge3/envs/probts/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to

Training: |          | 0/? [00:00<?, ?it/s]

tensor(0.6360, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(969.5607, device='mps:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(36.6321, device='mps:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(2.0905, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.7238, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.3883, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.8685, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.3971, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.8349, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.7804, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.5967, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.3151, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.9678, device='

In [ ]:
test_dataset = TSDatasetTest(data=test_data, context_length=context_length, scaler=train_loader.dataset.scaler)
test_loader = DataLoader(test_dataset, batch_size=int(1e6),shuffle=False)

In [ ]:
test_sample = next(iter(test_loader))

In [ ]:
test_sample.shape

In [ ]:
import numpy as np

#TODO: this is ugly, rewrite in future
def extract_forecasts(pred_samples, window, context_length):
    """
    Extracts and concatenates forecast windows from pred_samples.

    Args:
        pred_samples (np.ndarray): Array of prediction samples (e.g. [num_samples, forecast_len]).
        window (int): Number of forecast windows to extract.
        context_length (int): Number of context steps between each forecast start.

    Returns:
        np.ndarray: Concatenated forecast windows.
    """
    indices = [(i * (context_length + 1)) for i in range(window)]
    windows = [pred_samples[i][:context_length] for i in indices]
    return np.concatenate(windows)

In [ ]:
pred_samples = net.predict(test_sample, context_length)
pred_samples = pipeline.inverse_transform(pred_samples.numpy())

In [ ]:
pred  = extract_forecasts(pred_samples, 3, context_length)

In [ ]:
pred

In [ ]:
mae = np.mean(np.abs(pred - test_data[context_length:]))
print(f"MAE: {mae:.4f}")

In [ ]:
plt.plot(test_data[context_length:])
plt.plot(pred)

In [156]:
net.predict(test_sample, context_length)[0][10]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,